# Import Libraries

In [1]:
import tensorflow as tf
assert float(tf.__version__[:3]) >= 2.3

import os
import numpy as np
import glob
import time
import matplotlib.pyplot as plt
import time
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

# download data

In [2]:
!mkdir data

!wget  -O data/img0.JPG "https://d17fnq9dkz9hgj.cloudfront.net/breed-uploads/2018/08/siberian-husky-detail.jpg?bust=1535566590&width=630"
!wget  -O data/img1.JPG "https://www.hakaimagazine.com/wp-content/uploads/header-gulf-birds.jpg"
!wget  -O data/img2.JPG "https://www.artis.nl/media/filer_public_thumbnails/filer_public/00/f1/00f1b6db-fbed-4fef-9ab0-84e944ff11f8/chimpansee_amber_r_1920x1080.jpg__1920x1080_q85_subject_location-923%2C365_subsampling-2.jpg"
!wget  -O data/img3.JPG "https://www.familyhandyman.com/wp-content/uploads/2018/09/How-to-Avoid-Snakes-Slithering-Up-Your-Toilet-shutterstock_780480850.jpg"

--2022-01-27 12:59:11--  https://d17fnq9dkz9hgj.cloudfront.net/breed-uploads/2018/08/siberian-husky-detail.jpg?bust=1535566590&width=630
Resolving d17fnq9dkz9hgj.cloudfront.net (d17fnq9dkz9hgj.cloudfront.net)... 54.192.175.97, 54.192.175.115, 54.192.175.163, ...
Connecting to d17fnq9dkz9hgj.cloudfront.net (d17fnq9dkz9hgj.cloudfront.net)|54.192.175.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24112 (24K) [image/jpeg]
Saving to: ‘data/img0.JPG’

data/img0.JPG       100%[===================>]  23.55K  --.-KB/s    in 0.005s  

2022-01-27 12:59:11 (5.03 MB/s) - ‘data/img0.JPG’ saved [24112/24112]

--2022-01-27 12:59:11--  https://www.hakaimagazine.com/wp-content/uploads/header-gulf-birds.jpg
Resolving www.hakaimagazine.com (www.hakaimagazine.com)... 164.92.73.117
Connecting to www.hakaimagazine.com (www.hakaimagazine.com)|164.92.73.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 452718 (442K) [image/jpeg]
Saving to: ‘data/img1.JP

# Save resnet model

예제에 사용하는 resnet50 모델을 다운받아 saved_model 형식으로 저장합니다.

가중치는 imagenet 을 사용합니다.

In [3]:
model = ResNet50(weights='imagenet')
model.save('resnet50_saved_model') 

/home/keti/tf_2.5.0/lib/python3.6/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: resnet50_saved_model/assets


# Convertng saved model to tflite model

In [2]:
# A generator that provides a representative dataset
def representative_data_gen():
    dataset_list = glob.glob('./data/*')
    for i in range(100):
        image = next(iter(dataset_list))
        image = tf.io.read_file(image)
        
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])
        image = tf.cast(image / 255., tf.float32)
        image = tf.expand_dims(image, 0)
        yield [image]

# path to the SavedModel directory
converter = tf.lite.TFLiteConverter.from_saved_model('resnet50_saved_model') 

# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]  

 # This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen  

# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8] 

# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

# Save the model.
with open('resnet50.tflite', 'wb') as f:
    f.write(tflite_model)

# Test converted model

## test base resnet50 tflite model

In [3]:
import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter('resnet50.tflite' , 
                          experimental_delegates=[tflite.load_delegate('libedgetpu.so.1')])

interpreter.allocate_tensors()

### Test speed

In [4]:
# 2. input 값을 설정

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
img_path = 'data/img0.JPG'  # Siberian_husky
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
x = tf.constant(x)

input_shape = input_details[0]['shape']
input_data = np.array(x, dtype=np.uint8)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [5]:
for i in range(0, 50):
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    term = end_time - start_time
    print(f'term: {term} sec')

term: 0.7660369873046875 sec
term: 0.7443368434906006 sec
term: 0.7415304183959961 sec
term: 0.7364492416381836 sec
term: 0.7397317886352539 sec
term: 0.7374796867370605 sec
term: 0.7354466915130615 sec
term: 0.7444286346435547 sec
term: 0.7341039180755615 sec
term: 0.7413420677185059 sec
term: 0.7370338439941406 sec
term: 0.7369136810302734 sec
term: 0.7333879470825195 sec
term: 0.7377409934997559 sec
term: 0.7296833992004395 sec
term: 0.7348015308380127 sec
term: 0.7292580604553223 sec
term: 0.732677698135376 sec
term: 0.7399327754974365 sec
term: 0.7408628463745117 sec
term: 0.7405843734741211 sec
term: 0.7362217903137207 sec
term: 0.7428665161132812 sec
term: 0.7329473495483398 sec
term: 0.7420308589935303 sec
term: 0.7382183074951172 sec
term: 0.7355027198791504 sec
term: 0.7346653938293457 sec
term: 0.7360749244689941 sec
term: 0.7385683059692383 sec
term: 0.7370834350585938 sec
term: 0.7302224636077881 sec
term: 0.738215446472168 sec
term: 0.7324955463409424 sec
term: 0.73514866

## test compiled tflite model

`edgetpu_compiler` 를 실행하여 모델을 컴파일한다. 컴파일이 성공하면 뒤에 `_edgetpu` 가 붙은 `tflite` 모델이 생성된다. 
예시로 `resnet50.tflite` 파일을 컴파일하면 `resnet50_edgetpu.tflite` 가 생성된다


```bash
$ edgetpu_compiler resnet50.tflite
Edge TPU Compiler version 2.0.291256449

Model compiled successfully in 1618 ms.

Input model: resnet50.tflite
Input size: 25.08MiB
Output model: resnet50_edgetpu.tflite
Output size: 24.87MiB
On-chip memory available for caching model parameters: 6.25MiB
On-chip memory used for caching model parameters: 6.25MiB
Off-chip memory used for streaming uncached model parameters: 18.43MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 77
Operation log: resnet50_edgetpu.log
See the operation log file for individual operation details.
```

In [7]:
'''
아래처럼 jupyterlab 에서 실행하면 command not found 오류가 발생한다.
ssh 접속창 또는 터미널에서 직접 실행해야한다.

'''
! edgetpu_compiler resnet50.tflite

/bin/bash: edgetpu_compiler: command not found


In [8]:
import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter('resnet50_edgetpu.tflite' , 
                          experimental_delegates=[tflite.load_delegate('libedgetpu.so.1')])
interpreter.allocate_tensors()

In [9]:
# 2. input 값을 설정

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
img_path = 'data/img0.JPG'  # Siberian_husky
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
x = tf.constant(x)

input_shape = input_details[0]['shape']
input_data = np.array(x, dtype=np.uint8)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [10]:
total = 0
for i in range(0, 50):
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    term = end_time - start_time
    total += term
    print(f'term: {term} sec')

term: 0.05910038948059082 sec
term: 0.04474925994873047 sec
term: 0.04481625556945801 sec
term: 0.04474234580993652 sec
term: 0.044895172119140625 sec
term: 0.04508852958679199 sec
term: 0.045134544372558594 sec
term: 0.04532432556152344 sec
term: 0.04476356506347656 sec
term: 0.04505157470703125 sec
term: 0.044956207275390625 sec
term: 0.04482221603393555 sec
term: 0.04490017890930176 sec
term: 0.04475283622741699 sec
term: 0.04395771026611328 sec
term: 0.04442477226257324 sec
term: 0.04492998123168945 sec
term: 0.044834136962890625 sec
term: 0.04517841339111328 sec
term: 0.04442310333251953 sec
term: 0.04395794868469238 sec
term: 0.044403076171875 sec
term: 0.04491138458251953 sec
term: 0.04483342170715332 sec
term: 0.044753074645996094 sec
term: 0.04500412940979004 sec
term: 0.04482007026672363 sec
term: 0.04496169090270996 sec
term: 0.044847726821899414 sec
term: 0.044615745544433594 sec
term: 0.04465985298156738 sec
term: 0.04471921920776367 sec
term: 0.04480743408203125 sec
term: